# Zajęcia 1.

Zakres:
* zapoznanie sie z 2 srodowiskami pracy (klaster Hadoop oraz środowisko dostepne na GCP.
* operacje na rozproszonym i obiektowym systemie plików (HDFS i GCS)
* wykorzystanie platformy Apache Spark do interaktywnych kwerend na danych

Języki:
* bash, Python

Krótka lista przydatnych poleceń bash: https://www.reddit.com/r/linux/comments/9rns12/some_linux_commands_cheatsheet/

## Klaster Hadoop


URL Jupyter na klastrze Instytutu Informatyki: https://zsibio.ii.pw.edu.pl/jupyter/

### Architektura klastra
* edge node: cdh00 (węzeł dostępowy)
* HDFS:
  * name node: cdh01
  * data nodes: cdh02-cdh05
* YARN
  * resource manager: cdh01
  * node managers: cdh02-cdh05
* Dystrybucja Hadoop: Hortonworks
* konsola administracyjna: http://cdh01:8080/


### Jupyter Notebook

* Interaktywny notatnik dostepny poprzez przeglądarkę.
* Służy do wpisywania poleceń w wybranych jezykach programowania oraz opis w tzw jezyku markdown.
* Polecenia/opis wpisujemy do komórek. Komórki są wykonywane przez tzw. kernel (np Python w określonej wersji)
* Działa w tryb edycji komórek + tryb wykonywania poleceń. (przejście poprzez ESC i ENTER)
* Podstawowe skróty klawiaturowe: 
  * Esc/Enter
  * strzalki
  * ctrl-enter/shift-enter
  * a/b/d
  * y/m 
* Moze wykonywać polecenia powłoki (bash) !  %%bash
* Notatnik zapisywany jest w formacie ipynb (IPythonNoteBook)
* Kolejnosc uruchomienia komórek moze byc rozna.
* Mozliwosc zatrzymania kernela i uruchomienia od nowa. 

Polecamy film: https://www.youtube.com/watch?v=HW29067qVWk



### Security
Klaster jest zabezpieczony przed niepowolanym dostepem.
Zazwyczaj dostep do klastra to: zalogowanie poprzez ssh do wezla dostepowego a potem wykonywanie polecen. 
My logujemy sie do Jupytera, w tle pobierany jest ticket dostepowy do odpowiednich zasobów klastra. Korzystajac ze zmiennych srodowiskowych i plikow konfiguracyjnych Jupyter wie jak polaczyc sie z menedzerem zasobow oraz namenodem HDFS.

Dostep do Jupyter jest publiczny i po zajeciach mozna z niego korzystac.

## Kopiowanie danych do swojego katalogu domowego

W katalogu `/data/local/datascience/data/` znajduje sie plik który bedzie nam dzisiaj słuzyl do pracy. Należy go skopiować do swojego katalogu domowego. 
Dane pochodzą z https://gdac.broadinstitute.org/ i zawierają dane z badaniami nad mutacjami typu CNV w obrebie genu BRAC2.


In [1]:
%%bash
cd                # przejdz do katalogu domowego
mkdir -p data     # stworz katalog data (i posrednie)
cd data           # przejdz do katalogu data 
cp /data/local/datascience/data/brca.txt .  # skopiuj plik z lokalizacji do bieżącej lokalizacji
ls -lah           # wylistuj zawartość bieżącego katalogu


total 380K
drwxr-x---  3 iagasz bdg-users 4.0K Dec  4 15:55 .
drwx------ 31 iagasz bdg-users 4.0K Dec 11 23:10 ..
-rw-r--r--  1 iagasz bdg-users 367K Dec 11 23:13 brca.txt
drwxr-x---  2 iagasz bdg-users 4.0K Mar 29  2020 covid_19


In [2]:
%%bash
head ~/data/brca.txt   # pokaż początek pliku
echo                   # pusta linia
tail ~/data/brca.txt   # pokaż koniec pliku 
echo
wc -l ~/data/brca.txt  # policz linie pliku

Sample	Chromosome	Start	End	Num_Probes	Segment_Mean
TCGA-A2-A0EU-10A-01D-A060-02	1	10209	2583075	NA	-0.210121164888764
TCGA-A2-A0EU-10A-01D-A060-02	1	2583076	249240606	NA	-0.101789555120838
TCGA-A2-A0EU-10A-01D-A060-02	2	10002	243189359	NA	-0.337963741969504
TCGA-A2-A0EU-10A-01D-A060-02	3	60175	162511435	NA	-0.0850647472883194
TCGA-A2-A0EU-10A-01D-A060-02	3	162511436	162626067	NA	0.42309157125509
TCGA-A2-A0EU-10A-01D-A060-02	3	162626068	197962416	NA	-0.0962225847981023
TCGA-A2-A0EU-10A-01D-A060-02	4	69223	191014397	NA	-0.0713350286479217
TCGA-A2-A0EU-10A-01D-A060-02	5	11770	180905246	NA	0.176064461471459
TCGA-A2-A0EU-10A-01D-A060-02	6	63815	171050932	NA	-0.0895478602860139

TCGA-BH-A0W5-01A-11D-A106-02	16	60001	35285758	NA	0.50100860888744
TCGA-BH-A0W5-01A-11D-A106-02	16	46499754	90294729	NA	-0.131450478937173
TCGA-BH-A0W5-01A-11D-A106-02	17	1	81195162	NA	-0.0320031758392927
TCGA-BH-A0W5-01A-11D-A106-02	18	10064	78017233	NA	-0.0384891054622891
TCGA-BH-A0W5-01A-11D-A106-02	19	70880	5911


Do tej pory korzystalismy z lokalnego systemu plików do ktorego ma dostep maszyna na ktora sie zalogowalismy (cdh00). Teraz zaczniemy korzystac z rozproszonego systemu plikow.

## HDFS

* Dane są rozproszone na wielu serwerach i dzielone na bloki o ustalonym rozmiarze. 
* Bloki zostają rozdystrybuowane pomiędzy węzłami. Mogą być replikowane między węzłami.
* Name Node - odpowiedzialny za koordynację, zarządzanie metadanymi plików i aktywny monitoring replikacji plików.
* Data Node - węzeł składujący zawartość plików.
* Dostęp do danych, poprzez interfejs commandline, lub API programistyczne

Zakres na dzisiejszym lab:
* zapis i odczyt danych z HDFS
* manipulacje na plikach i podstawowe komendy

Dla podstawowych polecen systemowych na plikach istnieja odpowiedniki polecen dla systemu HDFS
* `ls` -> `hdfs dfs -ls`
* `cp` -> `hdfs dfs -cp`
* `mv` -> `hdfs dfs -mv`
* `rm` -> `hdfs dfs -rm`

Pełna lista poleceń znajduje się na www: https://hadoop.apache.org/docs/r2.4.1/hadoop-project-dist/hadoop-common/FileSystemShell.html

### Listowanie swojego HOME na HDFS
Każdy ma swoj katalog domowy na HDFS


In [3]:
! hdfs dfs -ls    # wylistuj zawartość katalogu domowego (HDFS)

Found 15 items
drwx------   - iagasz hdfs           0 2020-05-22 02:00 .Trash
drwxr-xr-x   - iagasz hdfs           0 2019-01-24 13:04 .hiveJars
drwxr-xr-x   - iagasz hdfs           0 2020-05-29 09:05 .skein
drwxr-xr-x   - iagasz hdfs           0 2020-06-11 09:19 .sparkStaging
-rw-r--r--   2 iagasz hdfs 37944356001 2019-01-04 11:12 Homo_sapiens.GRCh38.dna.toplevel.fa
-rw-r--r--   2 iagasz hdfs       33156 2019-01-21 12:36 chr1.slice.tgp.vcf.gz
drwxr-xr-x   - iagasz hdfs           0 2020-03-29 14:17 data
drwxr-xr-x   - iagasz hdfs           0 2019-12-03 17:56 e_coli
drwxr-xr-x   - iagasz hdfs           0 2019-12-16 13:24 edugen
drwxr-xr-x   - iagasz hdfs           0 2019-12-31 14:17 external
drwxr-xr-x   - iagasz hdfs           0 2020-01-13 11:42 survey
drwxr-xr-x   - iagasz hdfs           0 2020-01-26 13:27 survey2
drwxr-xr-x   - iagasz hdfs           0 2019-02-15 12:12 tgp_subset
drwxr-xr-x   - iagasz hdfs           0 2019-07-18 21:13 varia
drwxr-xr-x   - iagasz hdfs           0 2019-1

Swoj katalog mozna tez sprawdzic podajac sciezke bezwgledna od katalogu glownego

In [4]:
! hdfs dfs -ls /user/${USER} # wylistuj zawartość katalogu domowego (HDFS)

Found 15 items
drwx------   - iagasz hdfs           0 2020-05-22 02:00 /user/iagasz/.Trash
drwxr-xr-x   - iagasz hdfs           0 2019-01-24 13:04 /user/iagasz/.hiveJars
drwxr-xr-x   - iagasz hdfs           0 2020-05-29 09:05 /user/iagasz/.skein
drwxr-xr-x   - iagasz hdfs           0 2020-06-11 09:19 /user/iagasz/.sparkStaging
-rw-r--r--   2 iagasz hdfs 37944356001 2019-01-04 11:12 /user/iagasz/Homo_sapiens.GRCh38.dna.toplevel.fa
-rw-r--r--   2 iagasz hdfs       33156 2019-01-21 12:36 /user/iagasz/chr1.slice.tgp.vcf.gz
drwxr-xr-x   - iagasz hdfs           0 2020-03-29 14:17 /user/iagasz/data
drwxr-xr-x   - iagasz hdfs           0 2019-12-03 17:56 /user/iagasz/e_coli
drwxr-xr-x   - iagasz hdfs           0 2019-12-16 13:24 /user/iagasz/edugen
drwxr-xr-x   - iagasz hdfs           0 2019-12-31 14:17 /user/iagasz/external
drwxr-xr-x   - iagasz hdfs           0 2020-01-13 11:42 /user/iagasz/survey
drwxr-xr-x   - iagasz hdfs           0 2020-01-26 13:27 /user/iagasz/survey2
drwxr-xr-x   - iag

Sprawdzmy katalog glowny na HDFS

In [6]:
!hdfs dfs -ls / # wylistuj zawartość katalogu głównego (HDFS)

Found 18 items
drwxr-xr-x   - iagasz hdfs            0 2019-09-27 20:47 /OAP-DATA-DIR
drwxr-xr-x   - hdfs   hdfs            0 2020-02-29 18:21 /apps
drwxr-xr-x   - hdfs   hdfs            0 2019-01-02 21:36 /atsv2
drwxr-xr-x   - iagasz hdfs            0 2019-01-08 15:09 /bdgz_platform
drwxr-xr-x   - mapred hadoop          0 2020-05-16 22:49 /data
drwxr-xr-x   - iagasz hdfs            0 2020-01-11 11:20 /edugen
drwxr-xr-x   - hdfs   hdfs            0 2019-01-02 21:35 /hdp
drwxrwxrwx   - iagasz hdfs            0 2019-08-02 16:07 /igap
drwx------   - livy   hdfs            0 2019-11-10 18:30 /livy2-recovery
drwxr-xr-x   - mapred hdfs            0 2019-01-02 21:35 /mapred
drwxr-xr-x   - iagasz hdfs            0 2019-10-08 15:22 /oap-data-dir
drwxr-xr-x   - iagasz hdfs            0 2019-07-29 15:30 /opt
drwxr-xr-x   - hdfs   hdfs            0 2019-01-02 21:35 /ranger
drwxr-xr-x   - hdfs   hdfs            0 2019-01-02 21:36 /services
drwxrwxrwx   - spark  hadoop          0 2020-12-11 23:14 /s

### **** ZADANIE 1 ****
Korzystajac z analogii do polecen znanych z lokalnego sytemu pliku oraz z dokumentacji polecen dla HDFS stworz w swoim katalogu domowym na HDFS katalog external a w nim katalog data. Sprawdz czy foldery istnieją.

### Dodanie pliku na HDFS

In [7]:
%%bash
hdfs dfs -put ~/data/brca.txt /user/${USER}/external/data  # dodaj plik do wskazanej lokalizacji na HDFS
hdfs dfs -ls /user/${USER}/external/data                   # wylistuj zawartość katalogu na HDFS

Found 1 items
-rw-r--r--   2 iagasz hdfs     375612 2020-01-11 14:15 /user/iagasz/external/data/brca.txt


put: `/user/iagasz/external/data/brca.txt': File exists


Jaki jest rozmiar pliku? Mozna skorzystac z przelacznika -h w ls

In [8]:
!hdfs dfs -ls -h /user/${USER}/external/data   # rozmiar pliku 

Found 1 items
-rw-r--r--   2 iagasz hdfs    366.8 K 2020-01-11 14:15 /user/iagasz/external/data/brca.txt


A calkowita wielkosc pliku? Polecenie `du`

In [9]:
!hdfs dfs -du -h /user/${USER}/external/data/brca.txt  # całkowita przestrzeń zajmowana przez plik (replikacja)

366.8 K  733.6 K  /user/iagasz/external/data/brca.txt


Informacje o statusie pliku: polecenie `fsck`

In [10]:
!hdfs fsck /user/${USER}/external/data/brca.txt

Connecting to namenode via http://cdh01.cl.ii.pw.edu.pl:50070/fsck?ugi=iagasz&path=%2Fuser%2Fiagasz%2Fexternal%2Fdata%2Fbrca.txt
FSCK started by iagasz (auth:KERBEROS_SSL) from /192.168.157.200 for path /user/iagasz/external/data/brca.txt at Fri Dec 11 23:15:21 CET 2020

Status: HEALTHY
 Number of data-nodes:	4
 Number of racks:		1
 Total dirs:			0
 Total symlinks:		0

Replicated Blocks:
 Total size:	375612 B
 Total files:	1
 Total blocks (validated):	1 (avg. block size 375612 B)
 Minimally replicated blocks:	1 (100.0 %)
 Over-replicated blocks:	0 (0.0 %)
 Under-replicated blocks:	0 (0.0 %)
 Mis-replicated blocks:		0 (0.0 %)
 Default replication factor:	2
 Average block replication:	2.0
 Missing blocks:		0
 Corrupt blocks:		0
 Missing replicas:		0 (0.0 %)

Erasure Coded Block Groups:
 Total size:	0 B
 Total files:	0
 Total block groups (validated):	0
 Minimally erasure-coded block groups:	0
 Over-erasure-coded block groups:	0
 Under-erasure-coded block groups:	0
 Unsatisfactory placeme

Plik mozna rowniez odczytac bezposrednio z HDFS:

In [11]:
! hdfs dfs -head /user/${USER}/external/data/brca.txt # odczytaj początek pliku

Sample	Chromosome	Start	End	Num_Probes	Segment_Mean
TCGA-A2-A0EU-10A-01D-A060-02	1	10209	2583075	NA	-0.210121164888764
TCGA-A2-A0EU-10A-01D-A060-02	1	2583076	249240606	NA	-0.101789555120838
TCGA-A2-A0EU-10A-01D-A060-02	2	10002	243189359	NA	-0.337963741969504
TCGA-A2-A0EU-10A-01D-A060-02	3	60175	162511435	NA	-0.0850647472883194
TCGA-A2-A0EU-10A-01D-A060-02	3	162511436	162626067	NA	0.42309157125509
TCGA-A2-A0EU-10A-01D-A060-02	3	162626068	197962416	NA	-0.0962225847981023
TCGA-A2-A0EU-10A-01D-A060-02	4	69223	191014397	NA	-0.0713350286479217
TCGA-A2-A0EU-10A-01D-A060-02	5	11770	180905246	NA	0.176064461471459
TCGA-A2-A0EU-10A-01D-A060-02	6	63815	171050932	NA	-0.0895478602860139
TCGA-A2-A0EU-10A-01D-A060-02	7	10128	159128640	NA	-0.0402976850808555
TCGA-A2-A0EU-10A-01D-A060-02	8	23380	1238776	NA	-1.29512163093905
TCGA-A2-A0EU-10A-01D-A060-02	8	1238777	1246507	NA	0.484060599726464
TCGA-A2-A0EU-10A-01D-A060-02	8	1246508	1316313	NA	-1.34134440497319
TCGA-A2-A0EU-10A-01D-A060-02	8	1316314	3129168

### **** ZADANIE 2 ****
W swoim katalogu domowym stworz katalog external/temp. Skopiuj do niego plik brca.txt 

### Usuwanie i odzyskiwanie plikow

Do usuwanie sluzy polecenie -rm. Domyslnie usuwany plik jest przesuwany do kosza, skad mozna go odzyskac. 


In [12]:
%%bash
hdfs dfs -rm /user/${USER}/external/temp/brca.txt # usun plik 
hdfs dfs -ls /user/${USER}/external/temp/          # wylistuj zawartość katalogu

Found 1 items
-rwxr--rwx   2 iagasz hdfs     375612 2019-12-31 14:25 /user/iagasz/external/temp/brca.txt1577798651863


rm: `/user/iagasz/external/temp/brca.txt': No such file or directory


Dostalismy lokalizacje pliku z kosza, stamtad mozemy go skopiowac.

Do samodzielnej weryfikacji:
* Przywrócenie pliku z kosza
* Zachowanie polecenia rm przy podaniu parametru `skipTrash`.
* modyfikacja uprawnień na katalogu poprzez polecenie `chmod`

### Pobranie pliku z powrotem na lokalny dysk

In [13]:
!hdfs dfs -get /user/${USER}/external/data/brca.txt brca2.txt

## Apache Spark

* Apache Spark platforma ogólnego zastosowania, opensource, do przetwarzania duzych zbiorow danych.
* Posiada  API dla języków programowania: Scala, Python i R. 
* Przetwarzanie w Spark przetwarzanie jest wykonywane w większości  wprost w pamięci operacyjnej.
* Przeznaczenie: do uruchamiania  aplikacji i skryptów z wykorzystaniem uczenia maszynowego lub interaktywnych kwerend.
* Spark ten wspiera SQL (typ DataFrames), przetwarzanie strumieniowe oraz przetwarzanie grafów.
* Integracja z lokalną pamięci masową, rozproszonymi lub obiektowymi systemu plików.
* Spark można uruchamić na pojedynczej maszynie na środowisku klastrowym, lub w chmurze. 

Zakres na dzisiejsze laboratorium:
* stworzenie sesji Spark
* zaczytanie danych z pliku tekstowego
* kwerendy na danych

Na tych cwiczeniach bedziemy korzystac z pliku ktory zapisalismy na HDFS w pierwszej czesci cwicczenia.

In [14]:
! hdfs dfs -ls /user/${USER}/external/data # Sprawdzmy czy plik znajduje się w katalogu

Found 1 items
-rw-r--r--   2 iagasz hdfs     375612 2020-01-11 14:15 /user/iagasz/external/data/brca.txt


### Przygotowanie sesji Sparkowej
Losujemy numer portu. Domyslnie port na ktorym Spark wystawia swoj interfejs graficzny jest 4040. zeby nie zajmowac sobie wzajmnie numerow portu - wykonamy losowanie.
Nazwa aplikacji będzie udekorowana nazwą użytkownika który ją uruchamia.

In [15]:
import os
user_name = os.environ.get('USER')
print(user_name)

iagasz


In [16]:
import random
ui_port = random.randint(4000,4999)
print(ui_port)

4020


In [17]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.master('yarn-client') \
.config('spark.driver.memory','1g') \
.config('spark.executor.memory', '1g') \
.config('spark.ui.port',f'{ui_port}') \
.appName(f'app-{user_name}') \
.getOrCreate()

### Wczytanie danych


In [18]:
path = f'/user/{user_name}/external/data/brca.txt'

In [19]:
df = spark.read.load(path, format="csv", sep="\t", inferSchema="true", header="true")

### Charakterystyka danych

In [20]:
type (df)                 # jaki jest typ danych

pyspark.sql.dataframe.DataFrame

In [21]:
df.explain()              # fizyczny plan wykonania

== Physical Plan ==
*(1) FileScan csv [Sample#10,Chromosome#11,Start#12,End#13,Num_Probes#14,Segment_Mean#15] Batched: false, Format: CSV, Location: InMemoryFileIndex[hdfs://cdh01.cl.ii.pw.edu.pl:8020/user/iagasz/external/data/brca.txt], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Sample:string,Chromosome:int,Start:int,End:int,Num_Probes:string,Segment_Mean:double>


In [22]:
df.explain(True)          # logiczny i fizyczny plan wykonania

== Parsed Logical Plan ==
Relation[Sample#10,Chromosome#11,Start#12,End#13,Num_Probes#14,Segment_Mean#15] csv

== Analyzed Logical Plan ==
Sample: string, Chromosome: int, Start: int, End: int, Num_Probes: string, Segment_Mean: double
Relation[Sample#10,Chromosome#11,Start#12,End#13,Num_Probes#14,Segment_Mean#15] csv

== Optimized Logical Plan ==
Relation[Sample#10,Chromosome#11,Start#12,End#13,Num_Probes#14,Segment_Mean#15] csv

== Physical Plan ==
*(1) FileScan csv [Sample#10,Chromosome#11,Start#12,End#13,Num_Probes#14,Segment_Mean#15] Batched: false, Format: CSV, Location: InMemoryFileIndex[hdfs://cdh01.cl.ii.pw.edu.pl:8020/user/iagasz/external/data/brca.txt], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Sample:string,Chromosome:int,Start:int,End:int,Num_Probes:string,Segment_Mean:double>


In [23]:
df.rdd.getNumPartitions() # liczba partycji (bloków danych)

1

In [24]:
df.printSchema()          # schemat danych

root
 |-- Sample: string (nullable = true)
 |-- Chromosome: integer (nullable = true)
 |-- Start: integer (nullable = true)
 |-- End: integer (nullable = true)
 |-- Num_Probes: string (nullable = true)
 |-- Segment_Mean: double (nullable = true)



In [25]:
df.count()                # wymiary (liczba wierszy)

5298

In [26]:
len(df.columns)           # wymiary (liczba kolumn)

6

In [27]:
df.describe().show()      # pokaż podsumowanie danych w tabeli

+-------+--------------------+------------------+-------------------+-------------------+----------+------------------+
|summary|              Sample|        Chromosome|              Start|                End|Num_Probes|      Segment_Mean|
+-------+--------------------+------------------+-------------------+-------------------+----------+------------------+
|  count|                5298|              5298|               5298|               5298|      5298|              5298|
|   mean|                null|11.312948282370705|5.464405806266516E7|7.557261170932427E7|      null|0.2103169573704153|
| stddev|                null| 6.548797844563064|5.276515600857485E7|5.816178219098644E7|      null|0.8318029444477086|
|    min|TCGA-A2-A0EU-01A-...|                 1|                  1|              99263|        NA| -3.49124544908987|
|    max|TCGA-BH-A0W5-10A-...|                24|          249152065|          249240606|        NA|  4.25258455583762|
+-------+--------------------+----------

In [28]:
df.describe("Chromosome").show() # pokaż podsumowanie danych w kolumnie 

+-------+------------------+
|summary|        Chromosome|
+-------+------------------+
|  count|              5298|
|   mean|11.312948282370705|
| stddev| 6.548797844563064|
|    min|                 1|
|    max|                24|
+-------+------------------+



### Odczyt danych

In [29]:
df.show()                    # pokaż 20 wierszy danych 

+--------------------+----------+---------+---------+----------+-------------------+
|              Sample|Chromosome|    Start|      End|Num_Probes|       Segment_Mean|
+--------------------+----------+---------+---------+----------+-------------------+
|TCGA-A2-A0EU-10A-...|         1|    10209|  2583075|        NA| -0.210121164888764|
|TCGA-A2-A0EU-10A-...|         1|  2583076|249240606|        NA| -0.101789555120838|
|TCGA-A2-A0EU-10A-...|         2|    10002|243189359|        NA| -0.337963741969504|
|TCGA-A2-A0EU-10A-...|         3|    60175|162511435|        NA|-0.0850647472883194|
|TCGA-A2-A0EU-10A-...|         3|162511436|162626067|        NA|   0.42309157125509|
|TCGA-A2-A0EU-10A-...|         3|162626068|197962416|        NA|-0.0962225847981023|
|TCGA-A2-A0EU-10A-...|         4|    69223|191014397|        NA|-0.0713350286479217|
|TCGA-A2-A0EU-10A-...|         5|    11770|180905246|        NA|  0.176064461471459|
|TCGA-A2-A0EU-10A-...|         6|    63815|171050932|        NA|-

In [30]:
df.show(10, truncate=False) # pokaż 10 wierszy, nie skracaj danych

+----------------------------+----------+---------+---------+----------+-------------------+
|Sample                      |Chromosome|Start    |End      |Num_Probes|Segment_Mean       |
+----------------------------+----------+---------+---------+----------+-------------------+
|TCGA-A2-A0EU-10A-01D-A060-02|1         |10209    |2583075  |NA        |-0.210121164888764 |
|TCGA-A2-A0EU-10A-01D-A060-02|1         |2583076  |249240606|NA        |-0.101789555120838 |
|TCGA-A2-A0EU-10A-01D-A060-02|2         |10002    |243189359|NA        |-0.337963741969504 |
|TCGA-A2-A0EU-10A-01D-A060-02|3         |60175    |162511435|NA        |-0.0850647472883194|
|TCGA-A2-A0EU-10A-01D-A060-02|3         |162511436|162626067|NA        |0.42309157125509   |
|TCGA-A2-A0EU-10A-01D-A060-02|3         |162626068|197962416|NA        |-0.0962225847981023|
|TCGA-A2-A0EU-10A-01D-A060-02|4         |69223    |191014397|NA        |-0.0713350286479217|
|TCGA-A2-A0EU-10A-01D-A060-02|5         |11770    |180905246|NA       

In [31]:
df.select("Sample").show()  # pokaż tylko kolumne sample

+--------------------+
|              Sample|
+--------------------+
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
|TCGA-A2-A0EU-10A-...|
+--------------------+
only showing top 20 rows



In [32]:
df.select("Chromosome", "Start", "End").show() # pokaż kolumny chromosome, start i end

+----------+---------+---------+
|Chromosome|    Start|      End|
+----------+---------+---------+
|         1|    10209|  2583075|
|         1|  2583076|249240606|
|         2|    10002|243189359|
|         3|    60175|162511435|
|         3|162511436|162626067|
|         3|162626068|197962416|
|         4|    69223|191014397|
|         5|    11770|180905246|
|         6|    63815|171050932|
|         7|    10128|159128640|
|         8|    23380|  1238776|
|         8|  1238777|  1246507|
|         8|  1246508|  1316313|
|         8|  1316314|  3129168|
|         8|  3129169| 36339136|
|         8| 36339137| 39701358|
|         8| 39701359| 40086720|
|         8| 40086721| 48901482|
|         8| 48901483| 48911575|
|         8| 48911576|146303846|
+----------+---------+---------+
only showing top 20 rows



In [33]:
df_chrom = df.select(df.Chromosome).distinct() # stworz nowy DF zawierajacy tylko unikalne wartosci chromosomow

In [34]:
df_chrom.count()                     # policz wiersze w nowym DF

24

In [37]:
df.filter("Chromosome > 21").show() # pokaz dane spelniajace warunek

+--------------------+----------+--------+---------+----------+--------------------+
|              Sample|Chromosome|   Start|      End|Num_Probes|        Segment_Mean|
+--------------------+----------+--------+---------+----------+--------------------+
|TCGA-A2-A0EU-10A-...|        22|16972573| 51244552|        NA|  -0.459539615359244|
|TCGA-A2-A0EU-10A-...|        23| 2699504|154930285|        NA|   0.194146928345973|
|TCGA-A2-A0EU-01A-...|        22|16972573| 51244552|        NA|  -0.459539615359244|
|TCGA-A2-A0EU-01A-...|        23| 2699504|154930285|        NA|   0.194146928345973|
|TCGA-A7-A0D9-10A-...|        22|16051207| 51244552|        NA|  -0.226265745441125|
|TCGA-A7-A0D9-10A-...|        23| 2699504|154930285|        NA|  -0.186598214609633|
|TCGA-A7-A0D9-01A-...|        22|16051207| 51244552|        NA|  -0.226265745441125|
|TCGA-A7-A0D9-01A-...|        23| 2699504|154930285|        NA|  -0.186598214609633|
|TCGA-AO-A0JF-10A-...|        22|16051206| 51244552|        NA| 0

In [36]:
df.filter("Chromosome > 21").explain()  # pokaz plan wykonania

== Physical Plan ==
*(1) Project [Sample#10, Chromosome#11, Start#12, End#13, Num_Probes#14, Segment_Mean#15]
+- *(1) Filter (isnotnull(Chromosome#11) && (Chromosome#11 > 21))
   +- *(1) FileScan csv [Sample#10,Chromosome#11,Start#12,End#13,Num_Probes#14,Segment_Mean#15] Batched: false, Format: CSV, Location: InMemoryFileIndex[hdfs://cdh01.cl.ii.pw.edu.pl:8020/user/iagasz/external/data/brca.txt], PartitionFilters: [], PushedFilters: [IsNotNull(Chromosome), GreaterThan(Chromosome,21)], ReadSchema: struct<Sample:string,Chromosome:int,Start:int,End:int,Num_Probes:string,Segment_Mean:double>


In [ ]:
df.filter("Chromosome > 21 and Segment_Mean > 0").show() # pokaz dane spelniajace warunki

### Grupowanie i funkcje agregujące

In [38]:
from pyspark.sql.functions import *

df.groupBy("Chromosome").count().show()   # dokonaj grupowania po chromosomie i policz rekordy w grupie

+----------+-----+
|Chromosome|count|
+----------+-----+
|        12|  202|
|        22|  198|
|         1|  288|
|        13|  120|
|         6|  360|
|        16|  222|
|         3|  192|
|        20|  388|
|         5|  154|
|        19|  136|
|        15|  168|
|         9|  106|
|        17|  440|
|         4|  254|
|         8|  556|
|        23|  104|
|         7|  206|
|        10|  130|
|        24|    8|
|        21|   96|
+----------+-----+
only showing top 20 rows



In [39]:
df.groupBy("Chromosome").avg("Segment_Mean").show() # dokonaj grupowania po chromosomie i policz srednia wartosc segment_mean w grupie

+----------+--------------------+
|Chromosome|   avg(Segment_Mean)|
+----------+--------------------+
|        12|  0.2077776566255012|
|        22|-0.19456341730306023|
|         1|  0.2749020151479202|
|        13|-0.25463230407325826|
|         6| 0.09061564099997978|
|        16|  0.3809825220709315|
|         3|-0.02618245312138...|
|        20|  0.7508020639591368|
|         5|-0.04673099374977487|
|        19| 0.08623909396694052|
|        15|  0.6376181402358202|
|         9|-0.12425313098378711|
|        17| 0.24462914215524065|
|         4|-0.01256776781881...|
|         8|  0.6859064011278554|
|        23|-0.07294761544611646|
|         7| 0.17893302518868434|
|        10| 0.11082919688598039|
|        24| 0.07405671005723104|
|        21|-0.06047204925394...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# pogrupuj dane wzgledem probki i chromosomu, policz rekordy w grupie
df.groupBy("Sample","Chromosome").count().orderBy(asc("Sample")).show()

### Kolumny wyliczane

In [42]:
df.withColumn ("Length", col("End") - col("Start")).show() # dodaj kolumne dlugosc jako end - start

+--------------------+----------+---------+---------+----------+-------------------+---------+
|              Sample|Chromosome|    Start|      End|Num_Probes|       Segment_Mean|   Length|
+--------------------+----------+---------+---------+----------+-------------------+---------+
|TCGA-A2-A0EU-10A-...|         1|    10209|  2583075|        NA| -0.210121164888764|  2572866|
|TCGA-A2-A0EU-10A-...|         1|  2583076|249240606|        NA| -0.101789555120838|246657530|
|TCGA-A2-A0EU-10A-...|         2|    10002|243189359|        NA| -0.337963741969504|243179357|
|TCGA-A2-A0EU-10A-...|         3|    60175|162511435|        NA|-0.0850647472883194|162451260|
|TCGA-A2-A0EU-10A-...|         3|162511436|162626067|        NA|   0.42309157125509|   114631|
|TCGA-A2-A0EU-10A-...|         3|162626068|197962416|        NA|-0.0962225847981023| 35336348|
|TCGA-A2-A0EU-10A-...|         4|    69223|191014397|        NA|-0.0713350286479217|190945174|
|TCGA-A2-A0EU-10A-...|         5|    11770|1809052

In [44]:
df.withColumn ("Material", lit("DNA")).show()     # dodaj kolumne o stalej wartosci 'DNA'

+--------------------+----------+---------+---------+----------+-------------------+--------+
|              Sample|Chromosome|    Start|      End|Num_Probes|       Segment_Mean|Material|
+--------------------+----------+---------+---------+----------+-------------------+--------+
|TCGA-A2-A0EU-10A-...|         1|    10209|  2583075|        NA| -0.210121164888764|     DNA|
|TCGA-A2-A0EU-10A-...|         1|  2583076|249240606|        NA| -0.101789555120838|     DNA|
|TCGA-A2-A0EU-10A-...|         2|    10002|243189359|        NA| -0.337963741969504|     DNA|
|TCGA-A2-A0EU-10A-...|         3|    60175|162511435|        NA|-0.0850647472883194|     DNA|
|TCGA-A2-A0EU-10A-...|         3|162511436|162626067|        NA|   0.42309157125509|     DNA|
|TCGA-A2-A0EU-10A-...|         3|162626068|197962416|        NA|-0.0962225847981023|     DNA|
|TCGA-A2-A0EU-10A-...|         4|    69223|191014397|        NA|-0.0713350286479217|     DNA|
|TCGA-A2-A0EU-10A-...|         5|    11770|180905246|       

In [45]:
df.withColumn ("Chromosome2", concat(lit("chr"), col("Chromosome"))).show() # dodaj kolumne z konkatencja

+--------------------+----------+---------+---------+----------+-------------------+-----------+
|              Sample|Chromosome|    Start|      End|Num_Probes|       Segment_Mean|Chromosome2|
+--------------------+----------+---------+---------+----------+-------------------+-----------+
|TCGA-A2-A0EU-10A-...|         1|    10209|  2583075|        NA| -0.210121164888764|       chr1|
|TCGA-A2-A0EU-10A-...|         1|  2583076|249240606|        NA| -0.101789555120838|       chr1|
|TCGA-A2-A0EU-10A-...|         2|    10002|243189359|        NA| -0.337963741969504|       chr2|
|TCGA-A2-A0EU-10A-...|         3|    60175|162511435|        NA|-0.0850647472883194|       chr3|
|TCGA-A2-A0EU-10A-...|         3|162511436|162626067|        NA|   0.42309157125509|       chr3|
|TCGA-A2-A0EU-10A-...|         3|162626068|197962416|        NA|-0.0962225847981023|       chr3|
|TCGA-A2-A0EU-10A-...|         4|    69223|191014397|        NA|-0.0713350286479217|       chr4|
|TCGA-A2-A0EU-10A-...|        

In [46]:
df.drop("Chromosome").show()  # usun kolumne Chromosome. 
# Czy DF została pozbawiona kolumny na trwale?

+--------------------+---------+---------+----------+-------------------+
|              Sample|    Start|      End|Num_Probes|       Segment_Mean|
+--------------------+---------+---------+----------+-------------------+
|TCGA-A2-A0EU-10A-...|    10209|  2583075|        NA| -0.210121164888764|
|TCGA-A2-A0EU-10A-...|  2583076|249240606|        NA| -0.101789555120838|
|TCGA-A2-A0EU-10A-...|    10002|243189359|        NA| -0.337963741969504|
|TCGA-A2-A0EU-10A-...|    60175|162511435|        NA|-0.0850647472883194|
|TCGA-A2-A0EU-10A-...|162511436|162626067|        NA|   0.42309157125509|
|TCGA-A2-A0EU-10A-...|162626068|197962416|        NA|-0.0962225847981023|
|TCGA-A2-A0EU-10A-...|    69223|191014397|        NA|-0.0713350286479217|
|TCGA-A2-A0EU-10A-...|    11770|180905246|        NA|  0.176064461471459|
|TCGA-A2-A0EU-10A-...|    63815|171050932|        NA|-0.0895478602860139|
|TCGA-A2-A0EU-10A-...|    10128|159128640|        NA|-0.0402976850808555|
|TCGA-A2-A0EU-10A-...|    23380|  1238

### **** Zadanie 3 *****

a) Ile jest unikalnych danych próbek w tym zbiorze danych?

b) Pokaż nazwę próbki, numer chromosomu, początek i koniec segmentu dla segmentów występujących na chromosomie 21,22,23 i mających startową pozycję większą niż 10000000. 

c) Ile występuje wierszy dla każdej z próbki? Pokaż wynik z pełną nazwą próbki.

#### Przykładowe pobranie danych z DataFrame do zmiennej

In [57]:
a = df.collect()
len(a[1])

6

In [51]:
sample_number = df.orderBy("Sample").head().asDict()["Sample"] # Pobranie wartości do zmiennej
print (f"Numer probki to {sample_number}")

Numer probki to TCGA-A2-A0EU-01A-22D-A060-02


# Przeniesienie pliku na GCS

In [ ]:
! gsutil cp ~/data/brca.txt gs://bucket-agaszmurlo  # upload obiektu do kubełka